## Planificación de la producción: industria farmacéutica

V1.0, Nov 23

#### Problema CQM c/ reformulación Qubo


Un laboratorio farmacéutico tiene una línea de producción con dos medicamentos A y B, con
marca comercial y genérico respectivamente. Algunas características de su producción son: 

- entre los dos como máximo puede fabricar 10 unidades a la hora, pero no debiendo de bajar de las 4 unidades por hora en total.

- por motivos de política sanitaria, la producción de A ha de ser como mucho 2 unidades más que la de B.

- Cada unidad de tipo A vendida produce un beneficio de 60 euros, mientras que cada unidad de tipo B, genera un beneficio de 25 euros. 

1. Determinar las unidades de cada medicamento que deberá fabricar por hora para maximizar su beneficio
2. Obtener dicho beneficio


<b>Planteamiento algebraico:</b>

    - x1, x2 ; variables  A y  B

    - Función objetivo: minimizar{f(x1,x2)} = -60*x1 - 25*x2

- Restricciones:

    - x1 + x2 <= 10
    - x1 + x2 >= 4
    - x1 -x2 <= 2




In [151]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset,n=None):
    # Resultados
    
    subset=sampleset.filter(lambda s: s.is_feasible).aggregate()
    
    if not n:
        print(f"\nLas {len(subset)} soluciones factibles al problema son:\n")
    else:
        print(f"\nImprimiendo las {n}/{len(subset)} primeras soluciones factibles:\n")
    
    print(subset.slice(n))
    
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)

In [160]:
## Definición del problema CQM

x1 = Integer('Xa',upper_bound=10)
x2 = Integer('Xb',upper_bound=10)

cqm = ConstrainedQuadraticModel()


# Función objetivo: minimizar beneficio

cqm.set_objective(-60*x1 - 25*x2)

# Restricciones

cqm.add_constraint(x1 + x2 <= 10,"Producción 1: Xa + Xb <=10")
cqm.add_constraint(x1 + x2 >= 4,"Producción 2: Xb - Xa >= 4")
cqm.add_constraint(x1 - x2 <= 2,"Producción 3: Xa - Xb <= 2")
print("Planteamiento problema CQM")
print("==========================")
print(cqm)

Planteamiento problema CQM
Constrained quadratic model: 2 variables, 3 constraints, 8 biases

Objective
  -60*Integer('Xa') - 25*Integer('Xb')

Constraints
  Producción 1: Xa + Xb <=10: Integer('Xa') + Integer('Xb') <= 10.0
  Producción 2: Xb - Xa >= 4: Integer('Xa') + Integer('Xb') >= 4.0
  Producción 3: Xa - Xb <= 2: Integer('Xa') - Integer('Xb') <= 2.0

Bounds
  0.0 <= Integer('Xa') <= 10.0
  0.0 <= Integer('Xb') <= 10.0



### Solución por fuerza bruta

In [161]:
result = ExactCQMSolver().sample_cqm(cqm)

In [162]:
solCPU=sol_factibles(result,10)


Imprimiendo las 10/37 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0  6  4 -460.0       1 arra...    True
1  5  5 -425.0       1 arra...    True
2  5  4 -400.0       1 arra...    True
3  4  6 -390.0       1 arra...    True
4  5  3 -375.0       1 arra...    True
5  4  5 -365.0       1 arra...    True
6  3  7 -355.0       1 arra...    True
7  4  4 -340.0       1 arra...    True
8  3  6 -330.0       1 arra...    True
9  2  8 -320.0       1 arra...    True
['INTEGER', 10 rows, 10 samples, 2 variables]



<b>Resultados:</b>

- Medicamento A: 6 u/h y B: 4 u/h

- El beneficio /hora: 460€


### Solución con QPU

In [165]:
#reformulación qubo y sampling

fl=20 # factor penalizador de lagrange

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)

sampler = EmbeddingComposite(DWaveSampler())
result = sampler.sample(qubo, num_reads=1000)

In [166]:
sampleset=sol_problema(result)
solQPU=sol_factibles(sampleset,5)


Imprimiendo las 5/36 primeras soluciones factibles:

  Xa Xb energy num_oc. is_sat. is_fea.
0  6  4 -460.0      18 arra...    True
1  5  5 -425.0      21 arra...    True
2  5  4 -400.0      11 arra...    True
3  4  6 -390.0      48 arra...    True
4  5  3 -375.0      27 arra...    True
['INTEGER', 5 rows, 125 samples, 2 variables]


### <b>Analizando los resultados del sampler QPU</b>

- Se ha formulado un problema Qubo desde un problema CQM

- El espacio de soluciones factibles es similar al del solver CPU (36 vs 37)

- Se han replicado los valores óptimos obtenidos por fuerza bruta

- Con 1000 samples ya se obtuvieron modas > 15 para la función objetivo mínima 

- Con respecto al problema de las baterías, de complejidad estructural similar, resulta evidente que la menor dispersión del espacio de entrada reduce drásticamente el num_reads necesario. Sería interesante poder caracterizar esta relación.